In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
np.random.seed(42)

# «Проблемы качества и очистка данных»

In [4]:
# Загружаем данные
raw_df = pd.read_csv('rosbank_train.csv')
raw_df.head(5)

,PERIOD,cl_id,MCC,channel_type,currency,TRDATETIME,amount,trx_category,target_flag,target_sum
0,01/10/2017,0,5200,NaN,810,21OCT17:00:00:00,5023.0,POS,0,0.0
1,01/10/2017,0,6011,NaN,810,12OCT17:12:24:07,20000.0,DEPOSIT,0,0.0
2,01/12/2017,0,5921,NaN,810,05DEC17:00:00:00,767.0,POS,0,0.0
3,01/10/2017,0,5411,NaN,810,21OCT17:00:00:00,2031.0,POS,0,0.0
4,01/10/2017,0,6012,NaN,810,24OCT17:13:14:24,36562.0,C2C_OUT,0,0.0


In [5]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 490513 entries, 0 to 490512
Data columns (total 10 columns):
PERIOD          490513 non-null object
cl_id           490513 non-null int64
MCC             490513 non-null int64
channel_type    487603 non-null object
currency        490513 non-null int64
TRDATETIME      490513 non-null object
amount          490513 non-null float64
trx_category    490513 non-null object
target_flag     490513 non-null int64
target_sum      490513 non-null float64
dtypes: float64(2), int64(4), object(4)
memory usage: 37.4+ MB


In [6]:
raw_df = raw_df.drop('target_sum', axis=1)
raw_df.target_flag = raw_df.target_flag.astype(bool)

## 1. Формируем train и test выборку

In [7]:
# Составляем список из 1000 клиентов (случайным образом)
client_ids = set(np.random.choice(raw_df['cl_id'].unique(), size=1000,replace=False))

In [8]:
# Тестовая выборка
test_df = raw_df[raw_df.cl_id.isin(client_ids)].copy()

# Тренировочная выборка
train_df = raw_df[~raw_df.cl_id.isin(client_ids)].copy()
df = train_df

## 2. Создание производных фичей

### 2.1. Общая сумма транзакций по каждой из trx_category

In [9]:
df.groupby(['cl_id', 'trx_category']).agg({'amount': 'sum'}).unstack().fillna(0).reset_index().head()

cl_id   amount                                                   \
trx_category       BACK_TRX    C2C_IN   C2C_OUT CASH_ADV       CAT   DEPOSIT   
0                0      0.0       0.0  36562.00      0.0      0.00   20000.0   
1                1      0.0       0.0      0.00      0.0      0.00   95000.0   
2                5  13990.0       0.0  33878.38      0.0      0.00  130000.0   
3                9      0.0  296950.0      0.00      0.0      0.00    5000.0   
4               10      0.0   10000.0  21692.90      0.0  37694.98  426300.0   

                                                                
trx_category        POS WD_ATM_OTHER WD_ATM_PARTNER WD_ATM_ROS  
0               7821.00          0.0            0.0        0.0  
1             122578.04          0.0            0.0    50000.0  
2             290500.75       5500.0         6000.0    66500.0  
3              12365.09          0.0       100000.0   435000.0  
4             200956.11          0.0         5400.0   422300.0

### 2.2. Общая сумма транзакций по основным валютам

In [10]:
# Берем популярные Валюты(топ 3 популярных)
popular_mcc = list(df['currency'].value_counts(normalize=True).index[:3])
df[df['currency'].isin(popular_mcc)].groupby(['cl_id', 'currency']).agg({'amount': 'sum'}).unstack().fillna(0).reset_index().head()

cl_id      amount             
currency               810  840     978
0            0    64383.00  0.0    0.00
1            1   266693.13  0.0  884.91
2            5   544874.63  0.0   51.83
3            9   849315.09  0.0    0.00
4           10  1124343.99  0.0    0.00

### 2.3. Общая сумма транзакций по категориям MCC кодов

In [11]:
# Берем популярные MCC коды (топ 5 популярных)
popular_mcc = list(df['MCC'].value_counts(normalize=True).index[:5])
df[df['MCC'].isin(popular_mcc)].groupby(['cl_id', 'MCC']).agg({'amount': 'sum'}).unstack().fillna(0).reset_index().head()

cl_id    amount                                       
MCC            5411      5499     5812      5814      6011
0       0   2031.00      0.00     0.00      0.00   20000.0
1       1  14629.46    492.80  3180.01  82441.17  145000.0
2       5  44925.40   7196.70  3309.00    247.00  208000.0
3       9     39.00      0.00     0.00      0.00  540000.0
4      10  53920.36  13225.02   319.00   2010.00  854000.0

### 2.4. Траты в рабочие дни

In [12]:
def transaction_by_day_count(dataset):
    dataset['day_of_transaction'] = dataset['TRDATETIME'].apply( lambda x: (datetime.datetime.strptime(x, '%d%b%y:%H:%M:%S')).strftime('%w') ) 
    tmp_df = dataset.groupby(['cl_id','day_of_transaction'])['day_of_transaction'].size().unstack().fillna(0).reset_index()
    old_columns = [old_col for old_col in tmp_df.columns.tolist() if old_col in dataset['day_of_transaction'].unique()]
    tmp_df.rename(columns={old_col: 'day_' + old_col for old_col in old_columns}, inplace=True)
    return tmp_df

In [13]:
transaction_by_day_count(df).head()

day_of_transaction,cl_id,day_0,day_1,day_2,day_3,day_4,day_5,day_6
0,0,0.0,0.0,2.0,0.0,1.0,0.0,2.0
1,1,7.0,20.0,20.0,16.0,16.0,10.0,15.0
2,5,15.0,19.0,19.0,15.0,25.0,24.0,25.0
3,9,5.0,8.0,7.0,5.0,6.0,4.0,4.0
4,10,74.0,35.0,62.0,68.0,65.0,57.0,102.0
